In [2]:
import numpy as np
import sys
from sys import argv
import gdal, gdalconst
import math
# TODO: add directory crawler to be able to move through downloaded image sets (once they're downloaded)
# TODO: add identifier to array from folder name to associated files (whether it's landsat5,6,7,8)
# factor out different calculations based on satellite
# add nuvi calculations as other methods
# auto-clip input files (either to file or in memory) to aoi shapefile

# Type for internal calculations:
t = np.float64

red = gdal.Open("C:/Users/marioag/Documents/GitHub/telescope/python_notebooks/LC08_L1TP_007069_20180411_20180417_01_T1_sr_band4.tif")
nir = gdal.Open("C:/Users/marioag/Documents/GitHub/telescope/python_notebooks/LC08_L1TP_007069_20180411_20180417_01_T1_sr_band5.tif")
blue = gdal.Open("C:/Users/marioag/Documents/GitHub/telescope/python_notebooks/LC08_L1TP_007069_20180411_20180417_01_T1_sr_band2.tif")

geotiff = gdal.GetDriverByName('GTiff')
outfile_NDVI = "C:/Users/marioag/Documents/GitHub/telescope/python_notebooks/LC08_L1TP_007069_20180411_20180417_01_T1_sr_NDVI.tif"
outfile_EVI = "/Users/marioag/Dropbox (Personal)/MIT/MIT_F17/cau/remote_sensing/london/output/LC082010242017111701T1-SC20180213140543_ls8-20170421-etm-evi-2-4-5.tif"
outfile_SAVI = "/Users/marioag/Dropbox (Personal)/MIT/MIT_F17/cau/remote_sensing/london/output/LC082010242017111701T1-SC20180213140543_ls8-20170421-etm-savi-4-5.tif"


# Ugly syntax, but fast:
r = red.GetRasterBand(1).ReadAsArray(0, 0, red.RasterXSize, red.RasterYSize)
n = nir.GetRasterBand(1).ReadAsArray(0, 0, nir.RasterXSize, nir.RasterYSize)
b = blue.GetRasterBand(1).ReadAsArray(0, 0, blue.RasterXSize, blue.RasterYSize)

# Convert the 16-bit Landsat 8 values to floats for the division operation:
r = r.astype(t)
n = n.astype(t)

# Tell numpy not to complain about division by 0:
np.seterr(invalid='ignore')


tr = []
tr =  red.GetGeoTransform()
# print tr
# print type(tr)

def applyCorrection(x):
    xCor = (.00002*x) + -0.100000 
    print xCor
    xCor = xCor/math.sin(53.91758867)
    print xCor
    return xCor
    # open file
    # apply transformation as defined in file
    # return arrays to be used in ndvi function
    
def ndvi(r, n, outfile):
    ndvi = (n-r)/(n+r)
    output = geotiff.Create(outfile, red.RasterXSize, red.RasterYSize, 1, gdal.GDT_Float64)
    output.SetProjection(red.GetProjection())
    output.GetRasterBand(1).WriteArray(ndvi)
    return output

def evi(r,n,b, outfile):
    evi = 2.5* (((n - r)) / (n +6 * r - 7.5 * b - 1))
    output = geotiff.Create(outfile, red.RasterXSize, red.RasterYSize, 1, gdal.GDT_Float64)
    output.SetProjection(red.GetProjection())
    output.GetRasterBand(1).WriteArray(evi)
    return output

def savi(r,n, outfile):
    savi = (n - r) / (n + r + 0.5) * 0.5
    output = geotiff.Create(outfile, red.RasterXSize, red.RasterYSize, 1, gdal.GDT_Float64)
    output.SetProjection(red.GetProjection())
    output.GetRasterBand(1).WriteArray(savi)
    return output
    
applyCorrection(r)
# ndvi = ndvi(r,n, outfile_NDVI)
# ndvi.SetGeoTransform(tr)

# evi = evi(r,n,b, outfile_EVI)
# evi.SetGeoTransform(tr)

# savi = savi(r,n,outfile_SAVI)
# savi.SetGeoTransform(tr)

# The ndvi value is in the range -1..1, but we want it to be displayable, so:
# Make the value positive and scale it back up to the 16-bit range:
# ndvi = (ndvi + 1) * (2**15 - 1)

# And do the type conversion back:
# ndvi = ndvi.astype(np.uint16)
# output.SetProjection(red.GetProjection())

# output.GetRasterBand(1).WriteArray(ndvi)
# exit()

[[-0.29998 -0.29998 -0.29998 ... -0.29998 -0.29998 -0.29998]
 [-0.29998 -0.29998 -0.29998 ... -0.29998 -0.29998 -0.29998]
 [-0.29998 -0.29998 -0.29998 ... -0.29998 -0.29998 -0.29998]
 ...
 [-0.29998 -0.29998 -0.29998 ... -0.29998 -0.29998 -0.29998]
 [-0.29998 -0.29998 -0.29998 ... -0.29998 -0.29998 -0.29998]
 [-0.29998 -0.29998 -0.29998 ... -0.29998 -0.29998 -0.29998]]
[[0.61392635 0.61392635 0.61392635 ... 0.61392635 0.61392635 0.61392635]
 [0.61392635 0.61392635 0.61392635 ... 0.61392635 0.61392635 0.61392635]
 [0.61392635 0.61392635 0.61392635 ... 0.61392635 0.61392635 0.61392635]
 ...
 [0.61392635 0.61392635 0.61392635 ... 0.61392635 0.61392635 0.61392635]
 [0.61392635 0.61392635 0.61392635 ... 0.61392635 0.61392635 0.61392635]
 [0.61392635 0.61392635 0.61392635 ... 0.61392635 0.61392635 0.61392635]]


In [ ]:
### Arcpy snippet to clip all layers by the cleaned bounding geometry- just parking it here for reference, it was run from the arc console

path = ""

for file in os.listdir(path):
    if file.endswith(".tif"):
        out = path + '/clipped/' + file + "_clipped.tif"
        arcpy.Clip_management(file, rectangle, out, in_template_dataset="greenbelt_buffers_union", clipping_geometry="ClippingGeometry")
        

